In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.65


In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [3]:
# choose a name for the run history container in the workspace
#run_history_name = 'train-on-local'

# project folder
#project_folder = './sample_projects/train-on-local'

In [4]:
from azureml.core import Experiment
experiment_name = 'train-on-local'
exp = Experiment(workspace=ws, name=experiment_name)

In [5]:
#import os
#from azureml.core import Project

#project = Project.attach(workspace_object = ws,
#                         history_name = run_history_name,
#                         directory = project_folder)

#print(project.project_directory, project.history.name, sep = '\n')

In [6]:
import os
script_folder = './samples/train-on-local'
os.makedirs(script_folder, exist_ok=True)

In [7]:
%%writefile $script_folder/train.py

#Develop the model
import tensorflow as tf
import keras
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import wget

model = ResNet152(weights='imagenet')

#model.summary( )

wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

img_path = '220px-Lynx_lynx_poing.jpg'
#print(Image.open(img_path).size)
Image.open(img_path)

img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

preds = model.predict(img)
print('Predicted:', decode_predictions(preds, top=3))

Overwriting ./samples/train-on-local/train.py


In [8]:
#User-managed runs

#from azureml.core.runconfig import RunConfiguration

# Editing a run configuration property on-fly.
#run_config = RunConfiguration.load(project_object = project, run_config_name = "local")

#run_config.environment.python.user_managed_dependencies = True
#run_config.prepare_environment = True

# You can choose a specific Python environment by pointing to a Python path 
#run_config.environment.python.interpreter_path = '/home/ninghai/miniconda3/envs/sdk2/bin/python'

In [9]:
#User-managed runs

from azureml.core.runconfig import RunConfiguration

# Editing a run configuration property on-fly.
run_config_user_managed = RunConfiguration()

run_config_user_managed.environment.python.user_managed_dependencies = True

# You can choose a specific Python environment by pointing to a Python path 
#run_config.environment.python.interpreter_path = '/home/ninghai/miniconda3/envs/sdk2/bin/python'

In [10]:
#%%time 
#from azureml.core.run import Run

#run = Run.submit(project_object = project,
#                 run_config = run_config,
#                 script_to_run = 'train.py')

# Shows output of the run on stdout.
#run.wait_for_completion(show_output = True)

In [11]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, script='train.py', run_config=run_config_user_managed)
run = exp.submit(src)
run.wait_for_completion(show_output=True)

RunId: train-on-local_1538582934_03f2c04c

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Running: ['python', 'azureml-setup/run_script.py', 'python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'OutputCollection:context_managers.RunHistory', 'train.py']
Logging experiment running status in history service.
Uploading experiment status to history service.
Adding run profile attachment azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
2018-10-03 16:09:19.686575: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Predicted: [[('n02127052', 'lynx', 0.99590886), ('n02128385', 'leopard', 0.001150445), ('n02123159', 'tiger_cat', 0.0009417962)]]


The experiment completed success

{'runId': 'train-on-local_1538582934_03f2c04c',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-10-03T16:08:54.505997Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '46bfe1cf-4ca7-4514-8884-7f3a544bc0b1'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': [],
  'Framework': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': True,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2', {'pip': ['azureml-defaults']}]},
    'CondaDependenciesFile': None},
   'EnvironmentVariables': {'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'},
   'Docker': {'BaseImage': 'mcr.microsoft.com/azureml/base:0.1.3',
    'Enabled': False,
    'SharedVolumes': True,
    'GpuSupport': False,
    'Arguments': [],
    'BaseImageRegistry': {'Address': None,
     '

In [12]:
# make sure you have helpers.py in the current folder
#import helpers
#print(helpers.get_run_history_url(run))

In [13]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-local,train-on-local_1538582934_03f2c04c,azureml.scriptrun,Completed,Link to Azure Portal,Link to Documentation


In [14]:
# System managed runs
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Editing a run configuration property on-fly.
#run_config = RunConfiguration.load(project_object = project, run_config_name = "local")
run_config_system_managed = RunConfiguration()

# Use a new conda environment that is to be created from the conda_dependencies.yml file
#run_config.environment.python.user_managed_dependencies = False

# Automatically create the conda environment before the run
#run_config.prepare_environment = True

run_config_system_managed.environment.python.user_managed_dependencies = False
run_config_system_managed.prepare_environment = True

# add scikit-learn to the conda_dependencies.yml file
cd = CondaDependencies()
cd.add_conda_package('scikit-learn')
cd.add_conda_package('python=3.6')
cd.add_conda_package('tensorflow')
cd.add_conda_package('tornado==4.5.3')

cd.add_pip_package('papermill==0.14.1')
cd.add_pip_package('python-dotenv==0.9.0')
cd.add_pip_package('Pillow==5.2.0')
cd.add_pip_package('wget==3.2')
cd.add_pip_package('aiohttp==3.3.2')
cd.add_pip_package('toolz==0.9.0')
cd.add_pip_package('tqdm==4.23.4')
cd.add_pip_package('azure-cli==2.0.41')
#cd.add_pip_package('tensorflow')
cd.add_pip_package('keras==2.2.0')

#cd.save_to_file(project_dir = project_folder, conda_file_path = run_config.environment.python.conda_dependencies_file)

run_config_system_managed.environment.python.conda_dependencies = cd

In [15]:
#%%time 
#print("Submitting an experiment.")
#run = Run.submit(project_object = project,
#                 run_config = run_config, 
#                 script_to_run = "train.py")

# Shows output of the run on stdout.
#run.wait_for_completion(show_output = True)

In [16]:
src = ScriptRunConfig(source_directory=script_folder, script='train.py', run_config=run_config_system_managed)
run = exp.submit(src)
run.wait_for_completion(show_output = True)

RunId: train-on-local_1538583053_6c0e29aa

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Running ['conda', '--version']
Creating Conda environment...
Logging experiment preparation status in history service.
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.5
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base -c defaults conda



sqlite-3.25.2        |  1.9 MB | ########## | 100% 
scikit-learn-0.20.0  |  5.8 MB | ########## | 100% 
numpy-base-1.15.2    |  4.2 MB | ########## | 100% 
absl-py-0.5.0        |  146 KB | ########## | 100% 
mkl_fft-1.0.6        |  150 KB | ########## | 100% 
numpy-1.15.2         |   48 KB | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
  Using cached https://f

  Using cached https://files.pythonhosted.org/packages/3a/e1/5f9023cc983f1a628a8c2fd051ad19e76ff7b142a0faf329336f9a62a514/attrs-18.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/10/05f29904a073fdba2d367cdd8c7862260cc23cdee52ba0443f4599acbc93/multidict-4.4.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a7/85/601ef6484bf7a722daa76a4383c4ccfd4980b74ed6c2895392f53ed210d5/azure_cli_nspkg-3.0.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5a/20/65766a2b2a033453ae4fd9ac6ac0b02a9fd94a4ace948aad1c3643abc7d1/azure_cli_lab-0.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5f/9f/55b345c3f8fe5fffc1bb34d08c6fc1def81e7a16c01761e6fe5e77dd742c/azure_cli_advisor-0.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f1/70/642eb54b3f33b90c47acdb777a5f9613ff7904abf15216cb4a164f586160/azure_cli_profile-2.1.1-py2.py3-none-any.whl
  Using cac

  Using cached https://files.pythonhosted.org/packages/0c/5a/4a6a5378c7a86c3099670e2863a263ecec0eb58a3d4a50ba04bbe99cef18/azure_cli_find-0.2.12-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d5/00/629340da085c8f43d4552b26fa9acaa13fd817c7ba050228d8b9427a2844/azure_cli_feedback-2.1.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/6d/83aeb520dab78dbaeb396cb7cfd713365117aa0fc064018c1ec45be2b549/azure_cli_backup-1.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/80/20/02fd179a36abbcc056e17065d3d2314ae930de62468f5d63240fefabcb20/azure_cli_cdn-0.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3a/95/266c37f834299ef3da2e29253e9ed2d5b0a29a3ef1938978ab690a57682e/azure_cli_extension-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/31/c4eec7519ce93bdf490a4fc171e12f8201d986ff8075772cd8d3103b0a3d/azure_cli_eventgrid-0.2.0-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/49/de/0d69aedae7c5f6428314640b65947203ab80409c12b5d4e66fb5b7a4182e/azure_mgmt_keyvault-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/71/8273a7eeed0aff6a854237ab5453bc9aa67deb49df4832801c21f0ff3782/contextlib2-0.5.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bd/c9/6fdd990019071a4a32a5e7cb78a1d92c53851ef4f56f62a3486e6a7d8ffb/urllib3-1.23-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b1/e9/0a11f4feefb9c4c2f7e30d4d448e15b32df3adabe370be70aaf4278fbb52/azure_storage_nspkg-3.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/30/4e/27c34b62430286c6d59177a0842ed90dc789ce5d1ed740887653b898779a/pytz-2018.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/59/32/92cade62c645756a83598edf56289e9b19aae5370642a7ce690cd06bc72f/cryptography-2.3.1-cp34-abi3-manylinux1_x86_64.whl
  Using cached ht

  Using cached https://files.pythonhosted.org/packages/77/de/47e35a97b2b05c2fadbec67d44cfcdcd09b8086951b331d82de90d2912da/jsonschema-2.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/bc/9bd3b5c2b4774d5f33b2d544f1460be9df7df2fe42f352135381c347c69a/ipython_genutils-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/bb/a24838832ba35baf52f32ab1a49b906b5f82fb7c76b2f6a7e35e140bac30/decorator-4.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e6/b5a1de8b0cc4e07ca1b305a4fcc3f9806025c1b651ea302646341222f88b/pexpect-4.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/c9/cdeeeabc550848e2a07caa66cba28aa057d23b6feaa824ceafd32c3f2226/azure_cli_command_modules_nspkg-2.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/50/591adac2a6cd8d77cccd38bdf5d177ad6a2e95b9217daa9a60b5d86de5cd/azure_mgmt_managementgroups-0.1.0-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ed/4f/c322d72dc92dde543471bc23f9ac0eb6b9d0dc2441bb5ce938250040e307/azure_mgmt_trafficmanager-0.50.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/80/37/e80f577570b32648c4b88c8c48a46501a4868ae4c8d905774fd02c2b01fc/azure_keyvault-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/95/b451721e38ca0feddce03ee3ce86158e208d0150253bd371207a8df4e9c5/azure_mgmt_msi-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/cb/13502fdbaf520d08fb280eb31ecfe5d926b9cf92259c22280bbde96b307d/azure_mgmt_marketplaceordering-0.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d1/fd/8f4943670bcc71586b7440c3ee77bfe2e14a0c3340f3bbd6d8814e803ae2/azure_mgmt_media-1.0.0rc1-py2.py3-none-a

  Using cached https://files.pythonhosted.org/packages/42/a9/7e99652c6bc619d19d58cdd8c47560730eb5825d43a7e25db2e1d776ceb7/xmltodict-0.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/88/04/d1e49a53846807845fa54ef5d659af73b0280f5f799ca1212eb2284a5f24/azure_mgmt_recoveryservicesbackup-0.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/f2/ca2fc48cd37cd4bf33bd7553240dd066a3e404d4342f236674b822c29361/azure_mgmt_recoveryservices-0.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ab/17/1684f274bd57ff81b0ac9000030d5796bf88c9735a093ba8b693c39ca6fd/azure_mgmt_cdn-3.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6e/06/fafe8b5d881cfa68927e61557c8419dcfacb93e07f4ab17cc60959707a53/azure_mgmt_servicefabric-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/11/6828014c391b0c641ca94813b5c78cbf0984547252a7d71166594e074330/azure_mgmt_eventgrid

mkl-random 1.0.1 requires cython, which is not installed.
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.2.0 which is incompatible.
azure-cli-storage 2.2.2 has requirement azure-mgmt-storage==2.0.0rc4, but you'll have azure-mgmt-storage 3.0.0 which is incompatible.
azure-cli-sql 2.1.4 has requirement azure-mgmt-storage==2.0.0rc4, but you'll have azure-mgmt-storage 3.0.0 which is incompatible.
azure-mgmt-datalake-analytics 0.2.0 has requirement msrestazure~=0.4.7, but you'll have msrestazure 0.5.0 which is incompatible.
azure-cli-interactive 0.3.30 has requirement prompt-toolkit~=1.0.15, but you'll have prompt-toolkit 2.0.5 which is incompatible.
azure-cli-appservice 0.2.4 has requirement azure-mgmt-storage==2.0.0rc4, but you'll have azure-mgmt-storage 3.0.0 which is incompatible.
azure-mgmt-recoveryservicesbackup 0.1.1 has requirement msrest

#
# To activate this environment, use
#
#     $ conda activate /home/jayaubuntudsvm/.azureml/envs/azureml_4e8e0b8d265d6611c7aef506ffcc4196
#
# To deactivate an active environment, use
#
#     $ conda deactivate





Running: ['/home/jayaubuntudsvm/.azureml/envs/azureml_4e8e0b8d265d6611c7aef506ffcc4196/bin/python', 'azureml-setup/run_script.py', '/home/jayaubuntudsvm/.azureml/envs/azureml_4e8e0b8d265d6611c7aef506ffcc4196/bin/python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'OutputCollection:context_managers.RunHistory', 'train.py']
Logging experiment running status in history service.
Uploading experiment status to history service.
Adding run profile attachment azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
2018-10-03 16:16:50.620098: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled t

{'runId': 'train-on-local_1538583053_6c0e29aa',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-10-03T16:16:48.654932Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '46bfe1cf-4ca7-4514-8884-7f3a544bc0b1'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': [],
  'Framework': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6',
      {'pip': ['azureml-defaults',
        'papermill==0.14.1',
        'python-dotenv==0.9.0',
        'Pillow==5.2.0',
        'wget==3.2',
        'aiohttp==3.3.2',
        'toolz==0.9.0',
        'tqdm==4.23.4',
        'azure-cli==2.0.41',
        'keras==2.2.0']},
      'scikit-learn',
      'tensorflow',
      'tornado==4.5.3']},
    'Co

In [17]:
# import helpers.py
#import helpers
#print(helpers.get_run_history_url(run))

In [18]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-local,train-on-local_1538583053_6c0e29aa,azureml.scriptrun,Completed,Link to Azure Portal,Link to Documentation


In [19]:
# Docker based execution

from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

run_config_docker = RunConfiguration()

run_config_docker.environment.python.user_managed_dependencies = False
run_config_docker.prepare_environment = True
run_config_docker.environment.docker.enabled = True
run_config_docker.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

# Specify conda dependencies with scikit-learn
#cd = CondaDependencies.create(conda_packages=['scikit-learn'])

cd = CondaDependencies()
cd.add_conda_package('python=3.6')
cd.add_conda_package('tensorflow')
cd.add_conda_package('tornado==4.5.3')

cd.add_pip_package('papermill==0.14.1')
cd.add_pip_package('python-dotenv==0.9.0')
cd.add_pip_package('Pillow==5.2.0')
cd.add_pip_package('wget==3.2')
cd.add_pip_package('aiohttp==3.3.2')
cd.add_pip_package('toolz==0.9.0')
cd.add_pip_package('tqdm==4.23.4')
cd.add_pip_package('azure-cli==2.0.41')
cd.add_pip_package('keras==2.2.0')

run_config_docker.environment.python.conda_dependencies = cd

In [20]:
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, script='train.py', run_config=run_config_docker)
run = exp.submit(src)

In [21]:
#Get run history details
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-local,train-on-local_1538583610_4dee697f,azureml.scriptrun,Preparing,Link to Azure Portal,Link to Documentation


In [22]:
run.wait_for_completion(show_output = True)

RunId: train-on-local_1538583610_4dee697f

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Logging experiment preparation status in history service.
Running: ['sudo', 'docker', 'build', '-f', 'azureml-setup/Dockerfile', '-t', 'azureml/azureml_90134752845af3fd7d56168862b69a3a', '.']
Sending build context to Docker daemon  207.9kB
Step 1/13 : FROM mcr.microsoft.com/azureml/base:0.1.3
0.1.3: Pulling from azureml/base
3b37166ec614: Already exists
504facff238f: Already exists
ebbcacd28e10: Already exists
c7fb3351ecad: Already exists
2e3debadcbf7: Already exists
515d5d7ec0fe: Pulling fs layer
d1a5075872b0: Pulling fs layer
23a043b1b768: Pulling fs layer
c178f03f40ec: Pulling fs layer
c178f03f40ec: Waiting
515d5d7ec0fe: Download complete
515d5d7ec0fe: Pull complete
d1a5075872b0: Verifying Checksum
d1a5075872b0: Download complete
c178f03f40ec: Verifying Checksum
c178f03f40ec: Download compl

  Running setup.py bdist_wheel for wget: started
  Running setup.py bdist_wheel for wget: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Running setup.py bdist_wheel for toolz: started
  Running setup.py bdist_wheel for toolz: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
  Running setup.py bdist_wheel for pyyaml: started
  Running setup.py bdist_wheel for pyyaml: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
  Running setup.py bdist_wheel for idna-ssl: started
  Running setup.py bdist_wheel for id

  Found existing installation: wheel 0.31.1
    Uninstalling wheel-0.31.1:
      Successfully uninstalled wheel-0.31.1

#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_8ce184cd11f1d91f25e867d477394985
#
# To deactivate an active environment, use:
# > source deactivate
#

Removing intermediate container d2a15ab0300f
 ---> 907c8c48494a
Step 9/13 : ENV PATH /azureml-envs/azureml_8ce184cd11f1d91f25e867d477394985/bin:$PATH
 ---> Running in ae2fb3b2c742
Removing intermediate container ae2fb3b2c742
 ---> ae894408c256
Step 10/13 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_8ce184cd11f1d91f25e867d477394985/lib:$LD_LIBRARY_PATH
 ---> Running in 1858062fc309
Removing intermediate container 1858062fc309
 ---> 9ebf08ba5869
Step 11/13 : COPY azureml-setup/spark_cache.py azureml-setup/log4j.properties /azureml-setup/
 ---> d6ef9ebe0010
Step 12/13 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit "--repositories" "https://mmlspark.azureedge.net/m

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
2018-10-03 16:26:18.399024: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-10-03 16:26:18.400699: I tensorflow/core/common_runtime/process_util.cc:69] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.

243187712/243179624 [==============================] - 3s 0us/step

40960/35363 [==================================] - 0s 0us/step
Predicted: [[('n02127052', 'lynx', 0.99591225), ('n02128385', 'leopard', 0.0011504501), ('n02123159', 'tiger_cat', 0.00094180217)]]


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service

Execution Summary
RunId: train-on-local_15385836

{'runId': 'train-on-local_1538583610_4dee697f',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-10-03T16:26:15.356926Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '46bfe1cf-4ca7-4514-8884-7f3a544bc0b1'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': [],
  'Framework': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6',
      {'pip': ['azureml-defaults',
        'papermill==0.14.1',
        'python-dotenv==0.9.0',
        'Pillow==5.2.0',
        'wget==3.2',
        'aiohttp==3.3.2',
        'toolz==0.9.0',
        'tqdm==4.23.4',
        'azure-cli==2.0.41',
        'keras==2.2.0']},
      'tensorflow',
      'tornado==4.5.3']},
    'CondaDependenciesFile': 